# Applying PCA, then regressing

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

# ms = pd.read_feather("0326_mnist_binary/measures.feather")
# ms2 = pd.read_feather("0329_mnist_binary_gradient_norm/measures.feather")

ms_finetune = pd.read_feather("./0402_mnist_binary_finetune/measures.feather")
ms_finetune["kind"] = "IS: " + ms_finetune["model.init_scale"].map(str) + ", LR: " + ms_finetune["model.learning_rate"].map(str)

ms = ms_finetune 
# ms = pd.concat([ms2, ms_finetune], sort=False)
ms = ms.reset_index(drop=True)

print("Removing {} entries".format(sum(ms["gradient_norm_test"].isna())))
ms = ms[~ms["gradient_norm_test"].isna()]
ms["model.weights_product_reg_coef"] = ms["model.weights_product_reg_coef"].fillna(value=0)

smooth.analysis.remove_constant_columns(ms, verbose=True)

ms.head()

In [ ]:
ms["path_length_f_loss"] = ms["path_length_f_test"] - 1

In [ ]:
def plot_measure(measure_name):
    kinds = sorted(ms["kind"].unique())
    data = ms[[measure_name, "kind"]].copy()
    
    if "loss" in measure_name:
        log_measure_name = "log10_{}".format(measure_name)
        data[log_measure_name] = np.log10(ms[measure_name])
        measure_name = log_measure_name
    
    for kind in kinds:
        grid = sns.distplot(
            data.loc[data["kind"] == kind, measure_name],
            label=kind,
            hist_kws={
                "range": (data[measure_name].min(), data[measure_name].max()),
            },
            bins=10,
        )

    plt.legend()
    plt.show()

for measure in [
            "loss_train", "loss_test",
            "gradient_norm_test",
            "weights_product",
            "path_length_f_test",
            "path_length_f_loss",
            "path_length_d_test",
        ]:
    plot_measure(measure)

In [ ]:
dataset = smooth.datasets.from_params("mnist01")

import sklearn.decomposition
pca = sklearn.decomposition.PCA(n_components=2)
x2 = pca.fit_transform(dataset.x_train.reshape(len(dataset.x_train), -1))
# pca.transform(dataset.x_train[:1].reshape(1, -1))
x2

In [ ]:
dataset = smooth.datasets.from_params("mnist01", pca_dims=2)